In [1]:
import math
from pprint import pprint

one_item_time = 1e-3
n = 25

print('Worst case by simple fors in milenniums: ', (one_item_time * math.factorial(n))/(3600*24*365*1000))
print('Worst case by held-karp algorithm in days: ', one_item_time * (2**n*n**2)/(3600*24))

Worst case by simple fors in milenniums:  491857243890.5057
Worst case by held-karp algorithm in days:  242.72592592592594


In [2]:
file_content = '''+0.0 +0.0	Sunchales
+355.8 -405.9	Buenos Aires
-467.0 +274.3	Catamarca
+305.8 +383.6	Corrientes
-291.0 -53.7	Córdoba
+378.1 +528.2	Formosa
-415.1 +750.6	Jujuy
+402.2 -444.8	La Plata
-587.5 +170.5	La Rioja
-808.0 -216.8	Mendoza
-719.1 -891.4	Neuquén
+116.8 -89.0	Paraná
+632.0 +396.6	Posadas
+287.3 +387.3	Resistencia
-852.5 -2301.7	Río gallegos
-426.2 +683.8	Salta
-774.7 -66.7	San Juan
-531.9 -259.5	San Luis
+96.4 -77.8	Santa Fe
-302.1 -632.0	Santa Rosa
-300.2 +350.3	Santiago del Estero
-415.1 -1369.6	Trelew
-405.9 +457.8	Tucumán
-748.7 -2653.9	Ushuaia
-159.4 -1097.1	Viedma'''
from collections import namedtuple

City = namedtuple('City', 'id,name,x,y')

CITIES = []
for id_, line in enumerate(file_content.split('\n')):
    coord, name = line.split('\t')
    x, y = list(map(float, coord.split()))
    CITIES.append(City(id_, name, x, y))
pprint(CITIES)

PIQUETES = [(6, 15), (13, 3), (11, 18), (1, 7), (9, 16)]
PIQUETES.extend([(y, x) for x, y in PIQUETES])  # piquetes affects in both ways
pprint(['{} - {}'.format(CITIES[c1].name, CITIES[c2].name) for c1, c2 in PIQUETES])

[City(id=0, name='Sunchales', x=0.0, y=0.0),
 City(id=1, name='Buenos Aires', x=355.8, y=-405.9),
 City(id=2, name='Catamarca', x=-467.0, y=274.3),
 City(id=3, name='Corrientes', x=305.8, y=383.6),
 City(id=4, name='Córdoba', x=-291.0, y=-53.7),
 City(id=5, name='Formosa', x=378.1, y=528.2),
 City(id=6, name='Jujuy', x=-415.1, y=750.6),
 City(id=7, name='La Plata', x=402.2, y=-444.8),
 City(id=8, name='La Rioja', x=-587.5, y=170.5),
 City(id=9, name='Mendoza', x=-808.0, y=-216.8),
 City(id=10, name='Neuquén', x=-719.1, y=-891.4),
 City(id=11, name='Paraná', x=116.8, y=-89.0),
 City(id=12, name='Posadas', x=632.0, y=396.6),
 City(id=13, name='Resistencia', x=287.3, y=387.3),
 City(id=14, name='Río gallegos', x=-852.5, y=-2301.7),
 City(id=15, name='Salta', x=-426.2, y=683.8),
 City(id=16, name='San Juan', x=-774.7, y=-66.7),
 City(id=17, name='San Luis', x=-531.9, y=-259.5),
 City(id=18, name='Santa Fe', x=96.4, y=-77.8),
 City(id=19, name='Santa Rosa', x=-302.1, y=-632.0),
 City(id=20,

In [3]:
from functools import lru_cache

@lru_cache()
def distance(id1, id2):
    '''Return the euclidean distance between two cities'''
    if (id1, id2) in PIQUETES:
        return 1e12
    
    c1 = CITIES[id1]
    c2 = CITIES[id2]
    return math.sqrt((c1.x - c2.x)**2 + (c1.y - c2.y)**2)

assert distance(0, 0) == 0.0
assert round(distance(0, 1), 2) == 539.77
assert distance(1, 7) == 1e12 

In [45]:
def minimal_path(cities_set, last_city):
    best_path = None
    best_distance = None
    for city in cities_set:        
        path, dist = D(cities_set, city)        
        dist += distance(city, last_city)
        if best_distance is None or dist < best_distance:
            best_distance = dist
            best_path = path + (last_city,)
    return best_path, best_distance

def D(S, c):
    if len(S) == 1:
        return ((0, c), distance(0, c))
    else:
        new_S = S - set([c])
        return minimal_path(new_S, c)
        
def exact_TSP(cities):
    if len(cities) > 11:
        raise Exception('Too mach cities')
    initial = cities[0].id    
    S = set([x.id for x in cities[1:]])
    return minimal_path(S, 0)
    
exact_TSP(CITIES[:11])

((0, 1, 3, 5, 6, 2, 8, 4, 9, 10, 7, 0), 6355.38151906806)

In [37]:
import random

def is_neighbor(city, other_city):
    return city != other_city and (city, other_city) not in PIQUETES


def neighbors(city, not_visited):
    return [x for x in not_visited if is_neighbor(city, x)]


def compute_probabilities(city, not_visited, pheromone, alpha=1, beta=1):
    N = neighbors(city, not_visited)
    total = sum([pheromone[(city, destiny)] ** alpha * (1/distance(city, destiny)) ** beta 
                 for destiny in N])
    result = [
        (destiny, (pheromone[(city, destiny)] ** alpha * (1/distance(city, destiny)) ** beta)/total)  
        for destiny in N
    ]
    return result


def choose_with_probabilities(current_city, not_visited, pheromone):
    r = random.random()
    accum_prob = 0
    for city, prob in compute_probabilities(current_city, not_visited, pheromone):
        assert is_neighbor(current_city, city)
        accum_prob += prob
        if r <= accum_prob:
            return city
        
def path_cost(path):
    return sum(distance(c1, c2) for c1, c2 in zip(path[:-1], path[1:]))

        
class Ant:
    def __init__(self, cities):
        self.last_tour = []
        self.last_cost = 0
        self.cities = cities
    
    def make_tour(self, pheromone, initial_city=None):
        not_visited = self.cities[:]
        if initial_city is None:
            initial_city = random.choice(not_visited)
        else:
            initial_city = initial_city

        current_city = initial_city
        not_visited.remove(current_city)
        self.last_tour = [current_city]
        
        dead = False
        while not_visited and not dead:
            try:
                next_city = choose_with_probabilities(current_city, not_visited, pheromone)
                self.last_tour.append(next_city)
                not_visited.remove(next_city)        
                current_city = next_city
            except:
                dead = True
        if dead:
            self.last_tour = []
        else:
            if not is_neighbor(self.last_tour[-1], initial_city):
                self.last_tour = []
            else:
                self.last_tour.append(initial_city)
                self.last_cost = path_cost(self.last_tour)

In [133]:
def two_opt(current_path, current_cost):
    changes = True
    i = 0
    while changes and i<10:
        i+=1
        changes = False
        for i in range(1, len(current_path)-2):
            for j in range(i+1, len(current_path)-1):
                o = current_path[:]
                o[i], o[j] = o[j], o[i]
                c = path_cost(o)
                if c < current_cost:
                    current_cost = c
                    current_path = o
                    changes = True
    return current_path, current_cost

# initialize parameters
cities = [x.id for x in CITIES[:]]
current_pheromones = {(x, y): 1 for x in cities for y in cities if x != y}
ants = [Ant(cities) for x in range(100)]

iterations = []
avg_costs = []

iteration = 0
stagnation = False
while (iteration < 200 and not stagnation):
    # every ant makes his solution
    for a in ants:
        a.make_tour(current_pheromones)
        
    # update pheromone trails
    minimal_path_cost = 10e10
    delta_pheromones = {k: 0 for k in current_pheromones.keys()}
    
    total_cost = 0.0
    ants_no_dead = [x for x in ants if len(x.last_tour) > 0]
    selected_ants = sorted(ants_no_dead, key=lambda x: x.last_cost)[:10]
    for a in selected_ants:
        L = a.last_cost
        total_cost += L
        if L < minimal_path_cost:
            minimal_path_cost = L
        for c1, c2 in zip(a.last_tour[:-1], a.last_tour[1:]):
            try:
                delta_pheromones[(c1, c2)] += 1/L        
            except:
                print(a.last_tour)
                raise
    
    for k, delta in delta_pheromones.items():
        current_pheromones[k] = (0.8) * current_pheromones[k] + delta / len(selected_ants)
    
    iterations.append(iteration)
    avg_costs.append(total_cost / len(selected_ants))
    if iteration % 5 == 0:
        print('Iteration: {:>5} ## Minimal tour: {:.2f} ## AVG: {:.2f}'.format(iteration, minimal_path_cost, 
                                                                               avg_costs[-1]))

    stagnation = len(avg_costs) > 20 and all(avg_costs[-1] == x for x in avg_costs[-10:])
    if stagnation:
        print('stagnation!')
    iteration += 1

Iteration:     0 ## Minimal tour: 17774.12 ## AVG: 19545.83
Iteration:     5 ## Minimal tour: 17479.73 ## AVG: 18635.45
Iteration:    10 ## Minimal tour: 17983.71 ## AVG: 19231.33
Iteration:    15 ## Minimal tour: 17277.52 ## AVG: 19327.11
Iteration:    20 ## Minimal tour: 17978.98 ## AVG: 19022.94
Iteration:    25 ## Minimal tour: 17830.21 ## AVG: 19494.76
Iteration:    30 ## Minimal tour: 17884.40 ## AVG: 19125.24
Iteration:    35 ## Minimal tour: 17872.73 ## AVG: 19415.23
Iteration:    40 ## Minimal tour: 17166.17 ## AVG: 18268.30
Iteration:    45 ## Minimal tour: 15828.19 ## AVG: 17144.75
Iteration:    50 ## Minimal tour: 15043.24 ## AVG: 16418.00
Iteration:    55 ## Minimal tour: 13610.27 ## AVG: 14531.76
Iteration:    60 ## Minimal tour: 12515.12 ## AVG: 13565.87
Iteration:    65 ## Minimal tour: 11949.53 ## AVG: 12417.72
Iteration:    70 ## Minimal tour: 11685.93 ## AVG: 11901.89
Iteration:    75 ## Minimal tour: 11398.72 ## AVG: 11674.80
Iteration:    80 ## Minimal tour: 11523.

In [138]:
ant = ants[0]
ant.make_tour(current_pheromones, initial_city=0)
current_path = ant.last_tour
current_cost = ant.last_cost
print("Best result by ACO")
print(current_path, current_cost)
current_path, current_cost = two_opt(current_path, current_cost)
print("Result after 2-opt")
print(current_path, current_cost)

Best result by ACO
[0, 4, 19, 24, 21, 14, 23, 10, 9, 17, 16, 8, 2, 20, 15, 22, 6, 13, 5, 3, 12, 7, 11, 1, 18, 0] 11398.71761304764
Result after 2-opt
[0, 4, 19, 24, 21, 23, 14, 10, 9, 17, 16, 8, 2, 20, 15, 22, 6, 13, 5, 3, 12, 7, 11, 1, 18, 0] 11349.857956315562


In [90]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
import pandas as pd
output_notebook()
p = figure(title="AVG cost by iteration")
p.line(iterations, avg_costs)

show(p)

Loading BokehJS ...

In [57]:
Xs = [c.x for c in CITIES]
Ys = [c.y for c in CITIES]
names = [c.name for c in CITIES]
p= figure()
p.circle(Xs, Ys)
p.text(Xs, Ys, text=names, text_font_size="10pt")

for (c1, c2), pheromone in current_pheromones.items():
    x1, y1 = CITIES[c1].x, CITIES[c1].y
    x2, y2 = CITIES[c2].x, CITIES[c2].y
    if pheromone > 0.00001:
        p.segment(x1, y1, x2, y2, line_width=2)
show(p)